In [1]:
import os
import datetime as dt
import math
import numpy as np
import pandas as pd

In [2]:
# Input data files are available in the "../input/" directory.
input_dir = "../input"

questions = pd.read_csv(os.path.join(input_dir, 'questions.csv'))
questions['questions_date_added'] = pd.to_datetime(questions['questions_date_added'])

answers = pd.read_csv(os.path.join(input_dir, 'answers.csv'))
answers['answers_date_added'] = pd.to_datetime(answers['answers_date_added'])

comments = pd.read_csv(os.path.join(input_dir, 'comments.csv'))
comments['comments_date_added'] = pd.to_datetime(comments['comments_date_added'])

In [3]:
questions['questions_date_added'].min()

Timestamp('2011-09-27 15:23:17')

In [4]:
questions.head(3)

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....


In [5]:
answers.head(3)

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08,<p>Hi. I joined the Army after I attended coll...
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02,"<p>Dear Priyanka,</p><p>Greetings! I have answ..."


In [6]:
comments.head(3)

,comments_id,comments_author_id,comments_parent_content_id,comments_date_added,comments_body
0,f30250d3c2ca489db1afa9b95d481e08,9fc88a7c3323466dbb35798264c7d497,b476f9c6d9cd4c50a7bacdd90edd015a,2019-01-31 23:39:40,"First, you speak to recruiters. They are train..."
1,ca9bfc4ba9464ea383a8b080301ad72c,de2415064b9b445c8717425ed70fd99a,ef4b6ae24d1f4c3b977731e8189c7fd7,2019-01-31 20:30:47,Most large universities offer study abroad pro...
2,c354f6e33956499aa8b03798a60e9386,6ed20605002a42b0b8e3d6ac97c50c7f,ca7a9d7a95df471c816db82ee758f57d,2019-01-31 18:44:04,"First, I want to put you at ease that the oppo..."


**Questions**

In [7]:
questions['questions_text'] = questions['questions_title'] + ' ' + questions['questions_body']
questions_text = questions[['questions_id', 'questions_text']]
questions_text.head(5)

,questions_id,questions_text
0,332a511f1569444485cf7a7a556a5e54,Teacher career question What is a maths...
1,eb80205482e4424cad8f16bc25aa2d9c,I want to become an army officer. What can I d...
2,4ec31632938a40b98909416bdd0decff,Will going abroad for your first job increase ...
3,2f6a9a99d9b24e5baa50d40d0ba50a75,To become a specialist in business management...
4,5af8880460c141dbb02971a1a8369529,Are there any scholarships out there for stude...


In [8]:
questions_text.to_parquet('questions_text.parquet.gzip', compression='gzip')

**Collect Text Corpora for Training Period from September 2011 to June 2018 **

In [9]:
test_period_start = dt.datetime(2018, 7, 1)
test_period_end = dt.datetime(2019, 1, 31)

In [10]:
train_questions = questions[questions['questions_date_added'] < test_period_start]
train_questions['text'] = train_questions['questions_title'] + ' ' + train_questions['questions_body']
train_questions_by_users = train_questions.groupby('questions_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_questions_by_users = train_questions_by_users.rename(columns={'questions_author_id': 'user_id'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
train_answers= answers[answers['answers_date_added'] < test_period_start]
train_answers['text'] = train_answers['answers_body'].apply(str)
train_answers_by_users = train_answers.groupby('answers_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_answers_by_users = train_answers_by_users.rename(columns={'answers_author_id': 'user_id'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
train_comments = comments[comments['comments_date_added'] < test_period_start]
train_comments['text'] = train_comments['comments_body'].apply(str)
train_comments_by_users = train_comments.groupby('comments_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_comments_by_users = train_comments_by_users.rename(columns={'comments_author_id': 'user_id'})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**Direct Texts Which Are Actually Written by Users**

In [13]:
direct_user_texts = pd.concat([train_questions_by_users, train_answers_by_users, train_comments_by_users], axis=0)
direct_user_texts = direct_user_texts.groupby('user_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
print(direct_user_texts.shape)
direct_user_texts.head(3)
direct_user_texts.to_parquet('direct_user_texts.parquet.gzip', compression='gzip')

(19495, 2)


**Indirect Texts Forwarded from Interactions with Other Users**

In [14]:
train_questions_to_answers_by_users = train_questions.merge(
    train_answers[['answers_question_id', 'answers_author_id']], 
    left_on='questions_id', right_on='answers_question_id', how='inner').groupby(
    'answers_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_questions_to_answers_by_users = train_questions_to_answers_by_users.rename(columns={'answers_author_id': 'user_id'})
train_questions_to_answers_by_users.shape

(8090, 2)

In [15]:
train_answers_to_questions_by_users = train_answers.merge(
    train_questions[['questions_id', 'questions_author_id']], 
    left_on='answers_question_id', right_on='questions_id', how='inner').groupby(
    'questions_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_answers_to_questions_by_users = train_answers_to_questions_by_users.rename(columns={'questions_author_id': 'user_id'})
train_answers_to_questions_by_users.shape

(10416, 2)

In [16]:
train_questions_to_comments_by_users = train_questions.merge(
    train_comments[['comments_parent_content_id', 'comments_author_id']], 
    left_on='questions_id', right_on='comments_parent_content_id', how='inner').groupby(
    'comments_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_questions_to_comments_by_users = train_questions_to_comments_by_users.rename(columns={'comments_author_id': 'user_id'})
train_questions_to_comments_by_users.shape

(866, 2)

In [17]:
train_comments_to_questions_by_users = train_comments.merge(
    train_questions[['questions_id', 'questions_author_id']], 
    left_on='comments_parent_content_id', right_on='questions_id', how='inner').groupby(
    'questions_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_comments_to_questions_by_users = train_comments_to_questions_by_users.rename(columns={'questions_author_id': 'user_id'})
train_comments_to_questions_by_users.shape

(1266, 2)

In [18]:
train_answers_to_comments_by_users = train_answers.merge(
    train_comments[['comments_parent_content_id', 'comments_author_id']], 
    left_on='answers_id', right_on='comments_parent_content_id', how='inner').groupby(
    'comments_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_answers_to_comments_by_users = train_answers_to_comments_by_users.rename(columns={'comments_author_id': 'user_id'})
train_answers_to_comments_by_users.shape

(3232, 2)

In [19]:
train_comments_to_answers_by_users = train_comments.merge(
    train_answers[['answers_id', 'answers_author_id']], 
    left_on='comments_parent_content_id', right_on='answers_id', how='inner').groupby(
    'answers_author_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
train_comments_to_answers_by_users = train_comments_to_answers_by_users.rename(columns={'answers_author_id': 'user_id'})
train_comments_to_answers_by_users.shape

(2772, 2)

In [20]:
indirect_user_texts = pd.concat(
    [train_questions_to_answers_by_users, train_answers_to_questions_by_users,
     train_questions_to_comments_by_users, train_comments_to_questions_by_users,
     train_answers_to_comments_by_users, train_comments_to_answers_by_users], axis=0)
indirect_user_texts = indirect_user_texts.groupby('user_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
print(indirect_user_texts.shape)
indirect_user_texts.head(3)
indirect_user_texts.to_parquet('indirect_user_texts.parquet.gzip', compression='gzip')

(19205, 2)


**Merged User Texts**

In [21]:
merged_user_texts = pd.concat([direct_user_texts, indirect_user_texts], axis=0)
merged_user_texts = merged_user_texts.groupby('user_id')['text'].apply(lambda texts: '\n '.join(texts)).reset_index()
print(merged_user_texts.shape)
merged_user_texts.head(3)
merged_user_texts.to_parquet('merged_user_texts.parquet.gzip', compression='gzip')

(19495, 2)


**Latent Semantic Indexing**

In [23]:
direct_user_texts['text'][0]

'<p>Hi Mara. Photography can lead you into a wide range of other careers. For example, my sister-in-law was a working fashion photographer for a number of years but she also took work as a photo retoucher. That means she would digitally clean up and process images taken on other photo shoots to correct colors, add effects and even clean up blemishes on the models. She also taught photography for a number of years and even did some graphic design. If you hone your Photoshop skills you\'ll be in a good place to find all sorts of work in that arena. </p>\n<p>More importantly its critical that you remember that EVERY career path is open to you no matter what you choose first. If you want to do photography and act, or get a CPA certification and pursue law you can do any and all of it at the same time. Just be sure that when you are focusing on something, give it your undivided attention. I hope that helps.</p>\n <p>I think there is a lot of great advice here. I would just add one thing for

**NLP Functions**

In [26]:
import spacy
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')

from wordcloud import WordCloud

import gensim

In [24]:
token_pos = ['NOUN', 'VERB', 'PROPN', 'ADJ', 'INTJ', 'X']

def nlp_preprocessing(data):
    """ Use NLP to transform the text corpus to cleaned sentences and word tokens
    """    
    
    def token_filter(token):
        """ Keep tokens who are alphapetic, in the pos (part-of-speech) list and not in stop list
        """    
        return not token.is_stop and token.is_alpha and token.pos_ in token_pos
    
    processed_tokens = []
    data_pipe = nlp.pipe(data)
    for doc in data_pipe:
        filtered_tokens = [token.lemma_.lower() for token in doc if token_filter(token)]
        processed_tokens.append(filtered_tokens)
    return processed_tokens    

In [27]:
# Tokenize text
lsi_tokens = nlp_preprocessing(direct_user_texts['text'])

# Create vocabulary
remove_terms_below_document_numbers = 10
remove_terms_above_corpus_size = 0.5
vocabulary_size = 10000
lsi_dic = gensim.corpora.Dictionary(lsi_tokens)
lsi_dic.filter_extremes(no_below=remove_terms_below_document_numbers, 
                        no_above=remove_terms_above_corpus_size, 
                        keep_n=vocabulary_size)
lsi_dic.save('lsi_dic_size_{}.model'.format(vocabulary_size))

In [28]:
print('Dictionary size: {}'.format(len(lsi_dic)))

Dictionary size: 7922


In [29]:
# Convert tokens to gensim document format
lsi_corpus = [lsi_dic.doc2bow(doc) for doc in lsi_tokens]
# Apply tfidf on the corpus first to remove the effect of function words (those with high document frequencies but low tf-idf scores)
lsi_tfidf = gensim.models.TfidfModel(lsi_corpus)
lsi_tfidf.save('lsi_corpus_voc_size_{}.model'.format(vocabulary_size))
lsi_corpus = lsi_tfidf[lsi_corpus]

In [30]:
num_topics = 50
lsi_model = gensim.models.LsiModel(lsi_corpus, id2word=lsi_dic, num_topics=num_topics)
lsi_model.save('lsi_topics_{}.model'.format(num_topics))

**Test LSI Scores**

In [31]:
def compute_lsi_probs(doc, text_col, num_topics):
    doc_tokens = nlp_preprocessing([doc[text_col]])[0]
    doc_bow = lsi_tfidf[lsi_dic.doc2bow(doc_tokens)]
    doc_lsi = lsi_model[doc_bow]
    doc_scores = np.zeros(shape=num_topics)
    for v,k in doc_lsi:
        doc_scores[v] = k
    return doc_scores

def lsi_similarity(lsi_1, lsi_2):
    return np.dot(lsi_1,lsi_2) / np.sqrt(np.dot(lsi_1,lsi_1) * np.dot(lsi_2,lsi_2))

In [32]:
text_col = 'text'
doc1_scores = compute_lsi_probs(direct_user_texts.iloc[0], text_col, num_topics)
doc2_scores = compute_lsi_probs(direct_user_texts.iloc[0], text_col, num_topics)
lsi_similarity(doc1_scores, doc2_scores)

1.0

In [33]:
doc1_scores = compute_lsi_probs(direct_user_texts.iloc[0], text_col, num_topics)
doc2_scores = compute_lsi_probs(direct_user_texts.iloc[1], text_col, num_topics)
lsi_similarity(doc1_scores, doc2_scores)

0.1878751269803218

In [34]:
doc1_scores = compute_lsi_probs(direct_user_texts.iloc[0], text_col, num_topics)
doc2_scores = compute_lsi_probs(direct_user_texts.iloc[2], text_col, num_topics)
lsi_similarity(doc1_scores, doc2_scores)

0.6416145506631263

**Extract LSI Scores**

In [35]:
def compute_lsi_scores(doc, text_col, num_topics):
    doc_tokens = nlp_preprocessing([doc[text_col]])[0]
    doc_bow = lsi_tfidf[lsi_dic.doc2bow(doc_tokens)]
    doc_lsi = lsi_model[doc_bow]
    doc_scores = np.zeros(shape=num_topics)
    for v,k in doc_lsi:
        doc_scores[v] = k
    return pd.Series(doc_scores)

**LSI Scores for Questions**

In [36]:
print(questions_text.shape)
questions_topics = questions_text.apply(compute_lsi_scores, axis=1, text_col='questions_text', num_topics=num_topics)
print(questions_topics.shape)

(23931, 2)
(23931, 50)


In [37]:
questions_topics = questions_topics.set_index(questions_text['questions_id'])
questions_topics.tail(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
questions_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
b64ef9cf0d33475ca9c22bdb8749f5a8,0.123529,0.043712,0.051510,-0.034777,0.139289,-0.029777,-0.382807,-0.115484,0.140156,0.017036,0.053820,-0.014905,0.023409,0.007551,-0.010244,-0.016827,0.036720,-0.053215,0.095487,0.039886,0.031410,0.112158,0.026561,0.125907,0.124973,-0.033707,0.077455,-0.045855,0.016038,0.032287,0.020795,-0.002906,-0.004890,-0.022016,0.032377,-0.011155,-0.008138,-0.024351,0.024936,-0.002459,-0.034467,-0.036499,-0.087526,-0.048360,-0.176361,0.014389,0.012480,0.068878,0.027319,0.031954
1b1ebe9afc5046ccbb328704e88e5cc0,0.203964,-0.290687,-0.029351,0.065431,-0.069719,-0.205919,-0.000940,-0.051782,-0.065327,0.088700,-0.030899,-0.027227,0.038261,0.032227,0.027788,0.053759,0.071501,0.007514,-0.071202,0.055705,-0.062154,0.053745,-0.025668,0.053350,0.015576,0.055556,-0.020406,-0.028468,-0.036920,-0.032521,0.043081,-0.000713,0.009716,0.009293,-0.027517,0.001972,-0.001562,-0.020123,0.015824,0.043063,-0.006378,0.030744,0.011145,0.015905,0.015783,-0.032967,0.106369,0.015047,-0.002358,-0.004258


In [38]:
questions_topics.columns = ['Topic_{}'.format(i) for i in questions_topics.columns]
questions_topics.to_parquet('questions_topics_vs_{}_nt_{}.parquet.gzip'.format(vocabulary_size, num_topics), compression='gzip')

**LSI Scores for Merged User Texts**

In [39]:
print(merged_user_texts.shape)
merged_user_topics = merged_user_texts.apply(compute_lsi_scores, axis=1, text_col='text', num_topics=num_topics)
print(merged_user_topics.shape)

(19495, 2)
(19495, 50)


In [40]:
merged_user_topics = merged_user_topics.set_index(merged_user_texts['user_id'])
merged_user_topics.tail(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ffff97b3dc494767ac937a39396ee247,0.245761,0.050395,-0.104139,-0.028716,-0.009507,0.039639,0.003503,-0.083190,-0.187874,-0.135889,0.111366,-0.038715,0.042377,-0.018027,-0.018721,0.039878,-0.141538,-0.061823,0.050244,-0.056227,-0.098746,0.003026,0.072291,0.028189,-0.085298,-0.059618,-0.01224,-0.028424,0.021093,-0.013901,0.038107,-0.069571,-0.022382,-0.050718,0.024963,0.005431,0.013616,0.029504,0.030058,0.014468,-0.010014,0.008091,-0.016644,-0.015555,-0.059570,-0.058793,-0.006865,-0.037549,0.068487,-0.125607
ffffbe8d854a4a5a8ab1a381224f5b80,0.288650,0.021033,0.026470,-0.051697,0.088799,-0.000200,0.058483,-0.076089,-0.086747,0.003738,0.041048,0.003172,-0.038631,-0.019359,0.042410,-0.068837,0.084484,-0.004300,0.000373,0.028187,0.015947,-0.065934,-0.004799,-0.021142,0.072862,0.000398,-0.09534,-0.015880,0.016739,-0.069161,0.032481,0.031862,0.032164,0.018527,0.041916,-0.059618,-0.010822,-0.116634,-0.168366,-0.010147,0.001464,-0.140166,-0.153844,-0.067021,0.118526,-0.101487,-0.129664,0.025178,0.064966,-0.032721


In [41]:
merged_user_topics.columns = ['Topic_{}'.format(i) for i in merged_user_topics.columns]
merged_user_topics.to_parquet('merged_user_topics_vs_{}_nt_{}.parquet.gzip'.format(vocabulary_size, num_topics), compression='gzip')

In [42]:
os.listdir()

['__notebook_source__.ipynb',
 '.ipynb_checkpoints',
 'lsi_topics_50.model.projection',
 'lsi_corpus_voc_size_10000.model',
 'lsi_dic_size_10000.model',
 'merged_user_topics_vs_10000_nt_50.parquet.gzip',
 'questions_topics_vs_10000_nt_50.parquet.gzip',
 'indirect_user_texts.parquet.gzip',
 'merged_user_texts.parquet.gzip',
 'direct_user_texts.parquet.gzip',
 'lsi_topics_50.model',
 'questions_text.parquet.gzip']